In [1]:
import os.path as path
import numpy as np
import torch.nn.functional as functional
from IPython.display import display, clear_output
from ipywidgets import FloatSlider, interactive, VBox
import ipywidgets as widgets
import matplotlib.pyplot as plt

from constants import *
from data_loaders import *
from model import CapsNet

%matplotlib inline

DEBUG_MODE = False
USE_GPU = True
MODEL = "model111.pt" # Specifies which model to load
DATASET = "small_norb" # 'mnist', 'small_norb'
RECONSTRUCTION_TYPE = "FC" # 'FC' or 'Conv'

Re-run this block to reset your model outputs if you messed it up.

In [2]:
# Load model
if DATASET == "mnist":
    capsnet =  CapsNet(reconstruction_type=RECONSTRUCTION_TYPE, alpha=DEFAULT_ALPHA)
    _, test_loader = load_mnist(DEFAULT_BATCH_SIZE)
if DATASET == "small_norb":
    capsnet = CapsNet(reconstruction_type=RECONSTRUCTION_TYPE, alpha=DEFAULT_ALPHA, imsize=28, num_classes=5)
    _, test_loader = load_small_norb(DEFAULT_BATCH_SIZE)
if USE_GPU:
  capsnet.cuda()

model_path = path.join(SAVE_DIR, MODEL)
capsnet.load_state_dict(torch.load(model_path))

capsnet.eval()
data, target = iter(test_loader).next()
target = torch.eye(10).index_select(dim=0, index=target) # One-hot encode target
output, reconstruction, masked = capsnet(data.cuda())

Here is where you choose which input image to play around with.

In [4]:
i = np.random.randint(DEFAULT_BATCH_SIZE) # index of chosen image in last batch
capsules = output[i:i+1] # capsules that correspond to this specific image

# Find prediction
classes = torch.sqrt((capsules**2).sum(2))
classes = functional.softmax(classes, dim=1)
_, prediction = classes.max(dim=1)

if DEBUG_MODE:
    print("Image:{}".format(i))
    print("Target:{}".format(target[i:i+1,:].max(dim=1)[1].item()))
    print("Prediction:{}".format(prediction.item()))
    print(capsules[:,prediction,:,:].shape)

In [5]:
# Dirty work here
# TODO: Fix problems with capsules and prediction as parameters
def reconstruct(prediction,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15):
    capsules[:,prediction,0,:] = c0
    capsules[:,prediction,1,:] = c1
    capsules[:,prediction,2,:] = c2
    capsules[:,prediction,3,:] = c3
    capsules[:,prediction,4,:] = c4
    capsules[:,prediction,5,:] = c5
    capsules[:,prediction,6,:] = c6
    capsules[:,prediction,7,:] = c7
    capsules[:,prediction,8,:] = c8
    capsules[:,prediction,9,:] = c9
    capsules[:,prediction,10,:] = c10
    capsules[:,prediction,11,:] = c11
    capsules[:,prediction,12,:] = c12
    capsules[:,prediction,13,:] = c13
    capsules[:,prediction,14,:] = c14
    capsules[:,prediction,15,:] = c15
    
    reconstruction, _ = capsnet.decoder(capsules, data, target[i:i+1].cuda())
    
    im = np.squeeze(reconstruction.data.cpu().numpy())
    im += abs(im.min())
    im /= im.max()
    plt.subplot(1,2,1)
    plt.title("Reconstruction")
    plt.imshow(im, cmap="gray");
    im2 = data[i, 0].data.cpu().numpy()
    im2 += abs(im.min())
    im2 /= im.max()
    plt.subplot(1,2,2)
    plt.title("Input")
    plt.imshow(im2, cmap="gray");
    
def build_widgets(capsule_init):
        return interactive(reconstruct,
                prediction=prediction,
                c0=FloatSlider(description="Capsule 0",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[0]),
                c1=FloatSlider(description="Capsule 1",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[1]),
                c2=FloatSlider(description="Capsule 2",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[2]),
                c3=FloatSlider(description="Capsule 3",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[3]),
                c4=FloatSlider(description="Capsule 4",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[4]),
                c5=FloatSlider(description="Capsule 5",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[5]),
                c6=FloatSlider(description="Capsule 6",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[6]),
                c7=FloatSlider(description="Capsule 7",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[7]),
                c8=FloatSlider(description="Capsule 8",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[8]),
                c9=FloatSlider(description="Capsule 9",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[9]),
                c10=FloatSlider(description="Capsule 10",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[10]),
                c11=FloatSlider(description="Capsule 11",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[11]),
                c12=FloatSlider(description="Capsule 12",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[12]),
                c13=FloatSlider(description="Capsule 13",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[13]),
                c14=FloatSlider(description="Capsule 14",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[14]),
                c15=FloatSlider(description="Capsule 15",min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE, value=capsule_init[15]))

Currently all sliders are initialized to zeros, which means the initial reconstruction is not correct at all. You can set debug mode to true, and adjust the parameters according to the model output vector.

In [6]:
if DEBUG_MODE:
    print(capsules[:,prediction,:,:])

Re-run this block to reset capsule

In [7]:
MIN = -1
MAX = 1
STEP = 0.05
CONTINUOUS_UPDATE = True

# Initial values
capsule_init = capsules[:,prediction,:,:].squeeze()

w = build_widgets(capsule_init)
display(w)

interactive(children=(Dropdown(description='prediction', options=(tensor([ 0], device='cuda:0'),), value=tensor([ 0], device='cuda:0')), FloatSlider(value=-0.14005860686302185, description='Capsule 0', max=1.0, min=-1.0, step=0.05), FloatSlider(value=-0.05450993403792381, description='Capsule 1', max=1.0, min=-1.0, step=0.05), FloatSlider(value=0.08539066463708878, description='Capsule 2', max=1.0, min=-1.0, step=0.05), FloatSlider(value=-0.07287126779556274, description='Capsule 3', max=1.0, min=-1.0, step=0.05), FloatSlider(value=-0.16842655837535858, description='Capsule 4', max=1.0, min=-1.0, step=0.05), FloatSlider(value=0.05866212397813797, description='Capsule 5', max=1.0, min=-1.0, step=0.05), FloatSlider(value=-0.05401112884283066, description='Capsule 6', max=1.0, min=-1.0, step=0.05), FloatSlider(value=-0.19115768373012543, description='Capsule 7', max=1.0, min=-1.0, step=0.05), FloatSlider(value=0.28818586468696594, description='Capsule 8', max=1.0, min=-1.0, step=0.05), FloatSlider(value=0.13127805292606354, description='Capsule 9', max=1.0, min=-1.0, step=0.05), FloatSlider(value=0.028750568628311157, description='Capsule 10', max=1.0, min=-1.0, step=0.05), FloatSlider(value=0.43176063895225525, description='Capsule 11', max=1.0, min=-1.0, step=0.05), FloatSlider(value=0.04811880737543106, description='Capsule 12', max=1.0, min=-1.0, step=0.05), FloatSlider(value=-0.16286465525627136, description='Capsule 13', max=1.0, min=-1.0, step=0.05), FloatSlider(value=-0.10912024974822998, description='Capsule 14', max=1.0, min=-1.0, step=0.05), FloatSlider(value=-0.19226716458797455, description='Capsule 15', max=1.0, min=-1.0, step=0.05), Output()), _dom_classes=('widget-interact',))

In [8]:
# Experimental improvements for interaction with visualization
# CURRENTLY NOT WORKING

# def reconstruct(change, prediction, widgets_list):
#     for i, widget in enumerate(widgets_list):
#         capsules[:,prediction,i,:] = widget.value
    
#     reconstruction, _ = capsnet.decoder(capsules, data, target[i:i+1].cuda())
    
#     if DEBUG_MODE:
#         print(capsules)
#         print(target[i:i+1])
#         print(target[i:i+1].max(dim=1)[1].reshape(-1,1))
        
#     im = np.squeeze(reconstruction.data.cpu().numpy())
#     im += abs(im.min())
#     im /= im.max()
#     plt.subplot(1,2,1)
#     plt.title("Reconstruction")
#     plt.imshow(im, cmap="gray");
#     im2 = data[i, 0].data.cpu().numpy()
#     im2 += abs(im.min())
#     im2 /= im.max()
#     plt.subplot(1,2,2)
#     plt.title("Input")
#     plt.imshow(im2, cmap="gray");

# MIN = -1
# MAX = 1
# STEP = 1e-1
# CAPS_COUNT = 16
# CONTINUOUS_UPDATE = True

# # Credits to building these widgets: https://stackoverflow.com/questions/37622023
# widgets_list = []
# for i in range(CAPS_COUNT):
#     widgets_list.append(FloatSlider(description="Capsule "+str(i),
#                                     min=MIN, max=MAX, step=STEP, continuous_update=CONTINUOUS_UPDATE))
# for widget in widgets_list:
#     widget.observe(lambda change:reconstruct(change, prediction, widgets_list))
    
# w = VBox(children=widgets_list)